In [ ]:
!pip install langchain --quiet
!pip install unstructured --quiet
!pip install "unstructured[md]" --quiet
!pip install chromadb --quiet
!pip install -U boto3 --quiet 

In [ ]:
import boto3
import os
import json
import shutil
from typing import Dict, List

from langchain.document_loaders import UnstructuredMarkdownLoader, DirectoryLoader
from langchain.text_splitter import MarkdownTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

In [ ]:
STACK = 'LLMStack'
INPUT_DATA_LOCAL_DIR = 'input_data'

In [ ]:
def get_cf_config(stackname: str):
    cf_client = boto3.client('cloudformation')

    response = cf_client.describe_stacks(StackName=stackname)
    outputs = response["Stacks"][0]["Outputs"]

    cf_outputs = {}
    for i in outputs:
        cf_outputs[i['OutputKey']] = i['OutputValue']
    return cf_outputs

In [ ]:
stack_config = get_cf_config(STACK)

In [ ]:
def download_files_from_s3(bucket_name, dir_name: str = 'input_data'):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        
    s3_client = boto3.client('s3')
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    objs = bucket.objects.filter(Prefix=dir_name)
    for obj in objs:
        s3_client.download_file(bucket_name, obj.key, obj.key)

In [ ]:
download_files_from_s3(stack_config['S3BucketName'], INPUT_DATA_LOCAL_DIR)

In [ ]:
loader = DirectoryLoader(INPUT_DATA_LOCAL_DIR,
                         loader_cls=UnstructuredMarkdownLoader,
                         show_progress=True,
                         use_multithreading=True)

docs = loader.load()
print(f'Lazy loaded {len(docs)} docs.')

In [ ]:
md_splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = md_splitter.split_documents(docs)
print(f'There are {len(texts)} texts after splitting.')

In [ ]:
class ContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"text_inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["embedding"]

In [ ]:
content_handler = ContentHandler()

embeddings = SagemakerEndpointEmbeddings( 
    endpoint_name=stack_config['SageMakerEndpointEmbeddings'],
    region_name=stack_config['AWSRegion'],
    content_handler=content_handler
)

In [ ]:
chroma_dir = 'chroma'

if not os.path.exists(chroma_dir):
    os.makedirs(chroma_dir)
else:
    shutil.rmtree(chroma_dir)

doc_search = Chroma.from_documents(
    texts,
    embeddings,
    persist_directory=chroma_dir,
    collection_name='sagemaker_docs')

In [ ]:
# Save chroma db to S3
bucket_name = stack_config['S3BucketName']
s3 = boto3.client('s3')


def upload_to_s3(local_file, s3_bucket, s3_object_name):
    s3.upload_file(local_file, s3_bucket, s3_object_name)
    print(f"Uploaded {local_file} to {s3_bucket}/{s3_object_name}")


for root, dirs, files in os.walk(chroma_dir):
    for file in files:
        local_path = os.path.join(root, file)
        upload_to_s3(local_path, bucket_name, local_path)